## CNN的网络结构
CNN的结构和之前的神经网络结构很相似，对于前文介绍过的网络结构，因为相邻层的所有神经元都有连接，因此称为“**全连接**”（fully-connected）。网络结构可表示为：
![Affine层神经网络](http://www.ituring.com.cn/figures/2018/DeepLearning/116.png)

CNN的网络结构和上述结构非常相似，只是将Affine层替换为卷积层（Conv），并在ReLU层后面增加了池化层（Pooling），如下图所示：
![CNN神经网络](http://www.ituring.com.cn/figures/2018/DeepLearning/117.png)

## 卷积层
### 为什么使用卷积层？
Affine层在计算时，将数据的形状忽略掉（扁平化）。然而，图像通常都是有三个维度，高、长、通道。将图像的数据扁平化后，很多重要的空间信息可能被忽略了。卷积层则保持了这种多维度信息的原有形态。因为数据不再是一个序列，所以CNN网络的输入输出数据也被称为：**特征图**（feature map）

### 卷积运算
什么是卷积？在信号处理中，两个时间信号进行卷积，就相当于将一个信号在时间轴上平行移动，所经过的信号重叠区域进行相乘后叠加。
![image](http://dev-pic.oss-cn-beijing.aliyuncs.com/python/juanji.png)

对两个信号的时间序列做卷积运算，如果变换到频域，则对应频域上的乘积运算。因此，卷积运算相当于将输入信号通过了一个滤波器。

在进行卷积运算时，将滤波器在数据上平行移动，并将对应位置的乘积求和，即可得到卷积结果。

![image](http://www.ituring.com.cn/figures/2018/DeepLearning/118.png)

假设输入数据的矩阵形状是$H\times W$，滤波器矩阵形状是$FH\times FW$，则卷积后所得结果的矩阵形状应为$(H - FH + 1)\times (W - FW + 1)$。可以看到，因为$ FH > 0$且$FW > 0$，则卷积后矩阵变小了。

### 填充数据
在CNN中，通常各层的神经元数量是相等的，因此，在卷积之前可以扩充输入数据矩阵。幅度为P的填充，是指用幅度为P像素的0，填充矩阵四周。

![矩阵的填充](http://www.ituring.com.cn/figures/2018/DeepLearning/121.png)

填充后矩阵形状变为$(H + 2P)\times (W + 2P)$，卷积后矩阵形状变为$(H + 2P - FH + 1)\times (W + 2P - FW + 1)$。

### 步幅
步幅是指滤波器在数据矩阵中移动的单元数，默认是1，如果幅度变为2则乘积累加的次数将变少。卷积后矩阵的形状变为：

$$
OH = \frac{H + 2P - FH}{S} + 1 \\
OW = \frac{W + 2P - FW}{S} + 1 
$$

### 3维矩阵卷积
在2纬矩阵卷积中，并没有约束矩阵的具体形状。但是，对于三维矩阵卷积，要求两个矩阵的第三个维度上的形状是相同的。在第三个维度上，将所有二维矩阵的卷积进行直接累加。

### 图形化理解
上面的三维矩阵卷积可以理解为：
![image](http://www.ituring.com.cn/figures/2018/DeepLearning/125.png)

可以看到，矩阵的卷积后的结果将矩阵在第三个维度上“压平”了。为了使得第三个维度（通道）上，也具有多个卷积输出，则要使用多个滤波器，假定其数量为FN，则上图可表示为：
![image](http://www.ituring.com.cn/figures/2018/DeepLearning/127.png)

其中的偏置对于每一个滤波器而言是相同的。

### 批处理
和之前的mini-batch训练方法一样，一次处理N组数据可以提高训练的效率。卷积计算的示意图可修改为：
![image](http://www.ituring.com.cn/figures/2018/DeepLearning/128.png)

## 池化层
池化层本身的原理很简单，就是将一个区域（例如：2×2或者3×3等等）内的数据，取其中一个值来表达。如果取最大值，则成为Max池化；如果取均值，则称为Average池化。

池化有什么作用？本书中没有太多介绍，从原理上看，池化层可以降低数据数量，加速训练过程。同时，池化使得网络对数据的微小差异不敏感，具有鲁棒性。

![Max池化过程](http://www.ituring.com.cn/figures/2018/DeepLearning/129.png)

注意：池化过程中，图像通道数不发生改变。

## 卷积层和池化层的实现

### 基于img2col的展开
这里讲的是如何计算卷积，能够更好的提升效率。书中的图不一定好理解，这里做一个算法描述说明这个问题。以下图中要计算的卷积为例：
![image](http://www.ituring.com.cn/figures/2018/DeepLearning/124.png)

我们将第一行图中的灰色块定义为块0，第二行为块1，第三行为块2，第四行为块3。

### 卷积层的实现
#### 第一步：对1个通道上的展开计算块0
只看块0的第1个通道，卷积计算试讲数据快中的9个元素，与滤波器块中的9个元素对位相乘，乘积结果进行叠加。由于矩阵的乘法计算中，本身就包含对位相乘后叠加的计算（矩阵成绩结果第$i$行第$j$列，等于左矩阵第$i$行和右矩阵第$j$列对位乘积并求和），因此可以充分利用这一特点。

将数据里的块0拉平，从3×3变为1×9。将滤波器的3×3矩阵也拉平，只不过将其变为一列（9×1矩阵）。

这样两个向量计算乘积时，就相当于对块0做了卷积计算。

#### 第二步：对3个通道上的展开计算块0
有了上一步骤的结果，对3个通道进行处理就不难理解。3个通道也是对位乘积求和。因此，对于数据而言，在一行数据中继续接续通道2的9个块0数据，然后连接上通道3的9个数据即可。对于滤波器，则按照列的方向连接。滤波器展开后的列向量形状为：$FH\times FW \times C$行1列。

![image]( http://dev-pic.oss-cn-beijing.aliyuncs.com/python/%E7%90%86%E8%A7%A3im2col%E5%8D%B7%E7%A7%AF.jpg)

#### 第三步：对数据重复块1、块2、块3
对数据块1、块2、块3做同样的操作，并且将数据以行的形式接在上图蓝色数据下面，形成共计4行数据。这样，再做矩阵乘法时，就完成了卷积的计算。所以，左侧矩阵的行数应为总块数$OH\times OW$，列数为W矩阵元素数乘以通道数（$FH\times FW \times C$）。

![image](http://dev-pic.oss-cn-beijing.aliyuncs.com/python/%E7%90%86%E8%A7%A3im2col%E5%8D%B7%E7%A7%AF-4%E8%A1%8C.jpg)

经过上述3个步骤，就可以很简单地通过计算一个矩阵和一个向量的乘积，来完成卷积计算。

### 卷积层反向传播
#### 反函数col2img要做什么？
有了上述img2col的原理，反函数其实不难理解。
首先，仍然需要根据图像的形状、padding和步长，计算出OH和OW。
随后，可以将每一行的元素数除以通道数，获得每个块的数据，将每个块的数据reshape成FW列、FH行的块，然后填入image的对应块位置上即可。

在书中的例子中，比较巧妙的利用了reshape函数的作用。多维数组可能不好理解，单只要知道，reshape实质上就是在每个维度上按照顺序遍历。遍历到哪个元素，就将原有矩阵的当前顺位数据填充进去。就不难理解reshape的实现方法。

如果对应数据块来看，就是先经历FW列，经历FH次，得到第0块的1通道数据。随后遍历C个通道。然后是col的各行重复这个过程，先重复OW次得到前OW个块，重复OH遍，最终遍历所有快。

越是先经历的遍历，在reshape中的维度就越高。因此,col2img函数中，reshape的维度顺序就是$(OH, OW, C, FH, FW)$。如果，一次训练多个数据，则N的维度最低，应该将第一个维度设置成N，其他顺延。

### 池化层的实现
池化层也可以用img2col实现，实现时将块展开变成行。只不过，由于池化的时候，各个通道数据不需要求和，而是独立呈现。因此，使用img2col得到展开矩阵后，在将其变为只有$OH×OW$列的数据，这样就将各个通道“折叠”到了行上。随后求一行的最大值即可。

最后，按照先通道数C，后OW，最后OH的维度顺序，将求和后的向量变回矩阵即可。（这里OW和OH指的是池化后的矩阵形状，而不是卷积后的）。


## CNN的实现
有了上节中的卷积层（Conv）实现和池化层（Pooling）实现，CNN网络仍可以像全连接神经网络一样以层的形式实现。下图是一个3层CNN神经网络的典型例子：
![image](http://www.ituring.com.cn/figures/2018/DeepLearning/138.png)

图中，前面的一层为卷积+池化层，中间一层为全连接层，最后输出层使用了Softmax输出。实际应用中，卷几层、全连接层可能都不止一层，池化层也不是必须的，激活函数也可以考虑使用sigmoid函数。

### 实例中需要配置的参数
#### input_dim 输入数据维度
输入数据的通道数、高、宽三个维度(C, H, W)。

#### conv_param 卷积层超参数
主要包括：filter_num 滤波器数量、 filter_size 滤波器大小、 stride 步幅、pad 填充。这里滤波器是正方形的，所以宽高都是filter_size（前文中的FH和FW）。

#### hidden_size 和 output_size
隐藏层神经元数量和输出层神经元数量。

#### weitght_init_std 初始化时权重的标准差
初始化权值矩阵时，使用正态分布随机数，其标准差为weight_init_std。

### 关键代码

差别的部分主要在网络配置上，这里对初值的选取比较简单，直接使用了正态分布随机数。

```
self.params = {}

# W1和b1是卷积层的参数
self.params['W1'] = weight_init_std * \
                    np.random.randn(filter_num, input_dim[0],
                                    filter_size, filter_size)
self.params['b1'] = np.zeros(filter_num)

# W2和b2是全连接层的参数
self.params['W2'] = weight_init_std * \
                    np.random.randn(pool_output_size,
                                    hidden_size)
self.params['b2'] = np.zeros(hidden_size)

# W3和b3是输出层的参数
self.params['W3'] = weight_init_std * \
                    np.random.randn(hidden_size, output_size)
self.params['b3'] = np.zeros(output_size)
```
设置好参数后，可以将各层添加到layers中：
```
self.layers = OrderedDict()
self.layers['Conv1'] = Convolution(self.params['W1'],
                                   self.params['b1'],
                                   conv_param['stride'],
                                   conv_param['pad'])

self.layers['Relu1'] = Relu()
self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
self.layers['Affine1'] = Affine(self.params['W2'],
                              self.params['b2'])

self.layers['Relu2'] = Relu()
self.layers['Affine2'] = Affine(self.params['W3'],
                              self.params['b3'])
self.last_layer = softmaxwithloss()
```
### MNIST学习结果
对MNIST进行学习，训练序列和测试序列的学习精度大约都在99%左右，学习准确度已经非常高。
![image](http://dev-pic.oss-cn-beijing.aliyuncs.com/python/cnn_trainging.png)


## CNN的可视化
### 如何理解卷积层对图像的学习？

上文中第一个卷积层对图像的快速变化特征比较敏感，对于图像中的边缘部位容易形成匹配。能够学习到多少个特征点，和滤波器的数量有一定关系。

### 加深层的作用
如果有更多的层，每一层的输入是上一层输出的结果，是对特征数据的进一步学习。因此，可以在上一层基础上，构造出更大规模的特征图像，例如：文理、物体等。而最后的全连接层，本身就具有分类的能力，可以区分前面层中识别到得物体。

## 代表性的CNN
### 早期的LeNet
LeNet在1998年，但是主体结构和现在的CNN没有本质区别。只是激活函数使用的是sigmoid、池化层使用的是子采样。

![LeNet](http://www.ituring.com.cn/figures/2018/DeepLearning/142.png)

### AlexNet
AlexNet在2012年被提出，对深度学习性能产生了一次飞跃，这也是深度学习热潮开始的导火索。和LeNet相比，主要差异有：
1. 激活函数使用ReLU
2. 使用了局部正规化层LRN(Local Response Normalization)
3. 使用了Dropout
网络结构如下：
![AlexNet](http://www.ituring.com.cn/figures/2018/DeepLearning/143.png)

AlexNet具有多个卷几层结构，对图像的识别逐级深化：

![AlexNet对图像学习的逐级深化](http://www.ituring.com.cn/figures/2018/DeepLearning/141.png)

